## Determine LS atomic term symbol for specified orbital occupation

In [14]:
# No principal quantum number included
# KKI 8/13/2018
import numpy

In [15]:
# enter the number of electrons in each orbital type (spdf)
ns = 1
np = 0
nd = 9
nf = 0

In [16]:
occ = [ns, np, nd, nf]
orbstr = ['s', 'p', 'd', 'f']
Lstr = ['S', 'P', 'D', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N']
maxl = len(orbstr) - 1
maxL = len(Lstr) - 1

In [17]:
occstr = ''
for j in range(len(orbstr)):
    if occ[j] > 0:
        occstr += '{:s}{:d}'.format(orbstr[j], occ[j])
        if occ[j] > 2 * (2*j+1):
            print("You can't put {:d} electrons into the {:s} shell".format(occ[j], orbstr[j]))
            xit = 1/0
nelec = sum(occ)
print('{:d} electrons'.format(nelec))
print('Orbital occupation: {:s}'.format(occstr))

10 electrons
Orbital occupation: s1d9


In [18]:
def build_occlist(j, nelec, k=0, filled=None):
    # make a list of all possible occupation vectors for 
    # angular momentum = j, # of electrons = nelec
    # recursive: 'filled' is an occupation vector for previous electrons
    # 'k' is the index of the previous electron; the added electron must
    # be added later, i.e., with a higher index
    ncol = 2 * (2*j + 1)  # all combinations of spin (+- 1/2) and mj
    if nelec == 0:
        # no electrons at all
        return numpy.zeros(ncol)
    #
    #print('**', j, nelec, ncol, k, filled)
    #
    if filled is None:
        # no previous electrons
        filled = numpy.zeros(ncol)
    occlist = []
    # must leave room at the end for the other electrons
    imax = ncol - (nelec-1)
    for i in range(k, imax):
        if filled[i] == 0:
            # can put an electron here
            nplus1 = filled.copy()
            nplus1[i] = 1
            if nelec == 1:
                # this is the last electron
                occlist.append(nplus1.copy())
            else:
                # put the next electron anywhere remaining
                newocc = build_occlist(j, nelec-1, k=i+1, filled=nplus1)
                #print('nelec={:d}'.format(nelec), newocc)
                occlist.extend(newocc)
    return numpy.array(occlist)

In [19]:
# generate all possible occupation vectors
occv = []  # list of arrays
for l in range(maxl+1):
    occl = build_occlist(l, occ[l])
    occv.append(occl)
    #print('l = {:d}:\n'.format(l), occl)

In [20]:
# for each l, compute L and S for each occupation vector
ms = numpy.array([0.5, -0.5])  # array of ms values; same for all l
MLMS = []
for l in range(maxl+1):
    LSlist = []
    nvec = 1
    if len(occv[l].shape) == 2:
        nvec = occv[l].shape[0]
    print('{:d} vectors for l = {:d}'.format(nvec, l))
    # array of ml values
    ml = numpy.arange(-l, l+0.1, 1)
    # loop over occupation vectors
    for irow in range(nvec):
        # reshape vector to two rows; top is ms=-1/2
        if nvec == 1:
            # only one vector
            mvec = occv[l].reshape((2,-1))
        else:
            mvec = occv[l][irow].reshape((2, -1))
        ML = numpy.dot(mvec.sum(axis=0), ml)
        MS = numpy.dot(mvec.sum(axis=1), ms)
        LSlist.append([ML, MS])
    MLMS.append(numpy.array(LSlist))

2 vectors for l = 0
1 vectors for l = 1
10 vectors for l = 2
1 vectors for l = 3


In [21]:
# add up all combinations across shells (l values)
def MLMSproduct(MLMS, l=None, prev=[]):
    # recursive; need all combinations across l values
    # args: l = current value of l
    #       prev = list of existing (partial) products (ML, MS) pairs
    if l is None:
        # this is the top level; just call the next l down
        l = maxl
        prev = MLMS[l]
    else:
        # combine current list with previous
        newlist = []
        for pML in prev:
            for ML in MLMS[l]:
                newlist.append(pML + ML)
        prev = numpy.array(newlist)
    if l > 0:
        # recursive call to next lower l
        LStot = MLMSproduct(MLMS, l-1, prev)
        return LStot
    else:
        # this is the bottom (l=0)
        return prev

In [22]:
# add up all combinations across shells (l values)
MLMStot = MLMSproduct(MLMS)
nmicro = MLMStot.shape[0]
print('Found {:d} microstates for {:s}'.format(nmicro, occstr))

Found 20 microstates for s1d9


In [23]:
# encode the (ML, MS) pairs as complex numbers
cplx = numpy.dot(MLMStot, [1, 1j])

In [24]:
def create_symbol(L, S, jvals=True):
    # return a (string) term symbol
    term = '{:d}{:s}'.format(round(2*S + 1), Lstr[int(L)])
    if jvals:
        # include parenthetical values of J
        minj2 = round(2*abs(L - S))
        maxj2 = round(2*(L + S))
        jlist = []
        isodd = minj2 & 0x1
        for j2 in range(maxj2, minj2-1, -2):
            if isodd:
                jlist.append('{:d}/2'.format(j2))
            else:
                jlist.append('{:d}'.format(j2//2))
        term += '({:s})'.format(','.join(jlist))
    return term

In [25]:
# assign the states to term symbols
maxiter = 100
niter = 0
ntot = 0  # number of microstates accounted
symbols = []
while (cplx.shape[0] > 0) and (niter < maxiter):
    Smax = cplx.imag.max()
    imax = numpy.where(numpy.isclose(cplx.imag, Smax))
    Lmax = cplx[imax].real.max()
    term = create_symbol(Lmax, Smax)
    # delete the (ML, MS) pairs that correspond to this term
    compon = []
    for ML in numpy.arange(-Lmax, Lmax+0.1, 1):
        for MS in numpy.arange(-Smax, Smax+0.1, 1):
            compon.append(ML + MS * 1j)
    for c in compon:
        for ipair in range(cplx.shape[0]):
            if numpy.isclose(cplx[ipair], c):
                cplx = numpy.delete(cplx, ipair)
                break
    ntot += (2*Lmax + 1) * (2*Smax + 1)
    symbols.append(term)
    niter += 1
ntot = round(ntot)
print('Accounted for {:d} of {:d} microstates'.format(ntot, nmicro))
if ntot < nmicro:
    print('*** {:d} microstates are missing! ***'.format(nmicro - ntot))

Accounted for 20 of 20 microstates


In [26]:
print('Term symbols for {:s}:'.format(occstr), '\n\t'.join([''] + symbols))

Term symbols for s1d9: 
	3D(3,2,1)
	1D(2)
